In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
from rlway.pyosrd import OSRD

In [59]:
OSRD.use_cases

['cvg_dvg',
 'cvg_dvg_bug',
 'point_switch',
 'station_capacity2',
 'station_capacity2_bug']

In [60]:
sim = OSRD(use_case='cvg_dvg', dir='cvg_dvg')

[15:52:07,524] [INFO]  [StandaloneSimulationCommand] Loading infra: cvg_dvg/infra.json
[15:52:07,737] [INFO]              [FullInfra] parsing infra
[15:52:08,228] [INFO]              [FullInfra] adaptation to kotlin
[15:52:08,436] [INFO]              [FullInfra] loading signals
[15:52:08,524] [INFO]              [FullInfra] building blocks
[15:52:08,575] [INFO]  [StandaloneSimulationCommand] Loading input: cvg_dvg/simulation.json
[15:52:08,650] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.16
[15:52:09,061] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.17
[15:52:09,094] [INFO]  [StandaloneSimulationCommand] All simulations completed
[15:52:09,095] [INFO]  [StandaloneSimulationCommand] Saving results at: cvg_dvg/results.json


In [62]:
sim.route_tvds

{'rt.D0->D2a': 'CVG',
 'rt.buffer_stop.0->D0': 'D0<->buffer_stop.0',
 'rt.D1->D2a': 'CVG',
 'rt.buffer_stop.1->D1': 'D1<->buffer_stop.1',
 'rt.D2->buffer_stop.1': 'CVG',
 'rt.D2->buffer_stop.0': 'CVG',
 'rt.D2a->buffer_stop.4': 'DVG',
 'rt.D2a->buffer_stop.3': 'DVG',
 'rt.D3->D2': 'DVG',
 'rt.buffer_stop.3->D3': 'D3<->buffer_stop.3',
 'rt.D4->D2': 'DVG',
 'rt.buffer_stop.4->D4': 'D4<->buffer_stop.4'}